In [1]:
import mne
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report, write_raw_bids
import pandas as pd

%matplotlib qt

In [2]:
# Script to spilt the concatenated recording of subject 10 Dyskinesia protocol

bids_root = r"C:\Users\Jonathan\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\BIDS_01_Berlin_Neurophys\rawdata"

subject = 'EL010'
datatype = 'ieeg'
session = 'EcogLfpMedOffOnDys01'
task = 'DyskinesiaProtocol'
acquisition = 'StimOff'
run = '1'
suffix = 'ieeg'

bids_path = BIDSPath(subject=subject, session=session, task=task, acquisition=acquisition, run=run,
                     suffix=suffix, datatype=datatype, root=bids_root)

print(bids_path)
raw = read_raw_bids(bids_path=bids_path, verbose=False)
raw.plot()

C:/Users/Jonathan/Charité - Universitätsmedizin Berlin/Interventional Cognitive Neuromodulation - Data/BIDS_01_Berlin_Neurophys/rawdata/sub-EL010/ses-EcogLfpMedOffOnDys01/ieeg/sub-EL010_ses-EcogLfpMedOffOnDys01_task-DyskinesiaProtocol_acq-StimOff_run-1_ieeg.vhdr


C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/933072391.py:17: RuntimeWarning: Did not find any events.tsv associated with sub-EL010_ses-EcogLfpMedOffOnDys01_task-DyskinesiaProtocol_acq-StimOff_run-1.

The search_str was "C:\Users\Jonathan\Charité - Universitätsmedizin Berlin\Interventional Cognitive Neuromodulation - Data\BIDS_01_Berlin_Neurophys\rawdata\sub-EL010\**\ieeg\sub-EL010_ses-EcogLfpMedOffOnDys01*events.tsv"
  raw = read_raw_bids(bids_path=bids_path, verbose=False)
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/933072391.py:17: RuntimeWarning: MNI152NLin2009bAsym is not an MNE-Python coordinate frame for IEEG data and so will be set to 'unknown'
  raw = read_raw_bids(bids_path=bids_path, verbose=False)
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/933072391.py:17: RuntimeWarning: DigMontage is only a subset of info. There are 2 channel positions not present in the DigMontage. The required channels are:

['EEG_CZ_TM', 'EEG_FZ_TM'].

Consider using inst.set

Using qt as 2D backend.
Free RAM space could not be determined because"psutil" is not installed. Setting precompute to False.


In [3]:
# New 

# create table  for annotations
annot_table = []

#annot_table.field_names = ["task", "acquisition", "onset", "end"]

# 1 Rest 10:54-10:58
annot_table.append(['Rest','StimOffPreDopa',0,300])
# 2 Selfp li ???
annot_table.append(['SelfpacedHandTapL','StimOffPreDopa',312,1145])
# 3 Rest 3min 11:12 - ???
annot_table.append(['Rest','StimOffDopa00',1172,1340])

# 4 Selfp Li
# 5 Selp re
annot_table.append(['SelfpacedHandTapB','StimOffDopa05',1340,2005])
# 6 Rest 11:26 - 11:30
annot_table.append(['Rest','StimOffDopa15',2010,2220])

# 7 selfp Li 11:30 - 11:37
annot_table.append(['SelfpacedHandTapL','StimOffDopa20',2220,2792])

# 8 Free 11:37 - 11:48
annot_table.append(['Free','StimOffDopa25',2796,3323])

# 9 Rest 11:48-11:55
annot_table.append(['Rest','StimOffDopa35',3330,3719])

# 10 Selfp Li 11:55 - 12:01
annot_table.append(['SelfpacedHandTapL','StimOffDopa45',3750,4091])

# 11 Free 12:01-12:15
annot_table.append(['Free','StimOffDopa50',4097,5000])

# 12 Rest 12:15 - 12:20
annot_table.append(['Rest','StimOffDopa65',5000,5231])

# 13 Selfp Li 12:20 - 12:27
# 14 Selfp Re 12:27 - 12:30
annot_table.append(['SelfpacedHandTapB','StimOffDopa70',5234,5900])

# 15 Free 12:30 - 12:37
annot_table.append(['Free','StimOffDopa80',5910,6292])

# 16 Rest 12:38 - 12:43
annot_table.append(['Rest','StimOffDopa85',6296,6600])


annot_table = pd.DataFrame(annot_table, columns=["task", "acquisition", "onset", "end"])
annot_table['duration'] = annot_table['end'] - annot_table['onset']

In [4]:
annot_table

,task,acquisition,onset,end,duration
0,Rest,StimOffPreDopa,0,300,300
1,SelfpacedHandTapL,StimOffPreDopa,312,1145,833
2,Rest,StimOffDopa00,1172,1340,168
3,SelfpacedHandTapB,StimOffDopa05,1340,2005,665
4,Rest,StimOffDopa15,2010,2220,210
5,SelfpacedHandTapL,StimOffDopa20,2220,2792,572
6,Free,StimOffDopa25,2796,3323,527
7,Rest,StimOffDopa35,3330,3719,389
8,SelfpacedHandTapL,StimOffDopa45,3750,4091,341
9,Free,StimOffDopa50,4097,5000,903


In [5]:
# add the annotations to a copy and plot them for inspection
annot = mne.Annotations(onset=annot_table['onset'], duration=annot_table['duration'],
            description=annot_table['task']+annot_table['acquisition'])

raw2 = raw.copy().set_annotations(annot)
raw2.plot()

raws = raw2.crop_by_annotations(annotations=annot)


Free RAM space could not be determined because"psutil" is not installed. Setting precompute to False.


In [7]:
# write out to BIDS
i=0
for ra in raws:
    print(ra.info)
    
    bids_path = BIDSPath(subject=subject,
                        session="EcogLfpMedOffOnDys01",
                        task=annot_table['task'][i],
                        acquisition=annot_table['acquisition'][i],
                        run=run,
                        suffix=suffix,
                        datatype=datatype,
                        root=r"C:\Users\Jonathan\Documents\DATA\Dyskinesia")
    write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)
    i+=1

<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\README'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annotations descriptions: ['RestStimOffPreDopa']
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\ieeg\sub-EL010_ses-EcogLfpMedOffOnDys01_task-Rest_acq-StimOffPreDopa_run-1_events.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\dataset_description.json'...


C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-EL010_ses-EcogLfpMedOffOnDys01_task-Rest_acq-StimOffPreDopa_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annotatio

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-EL010_ses-EcogLfpMedOffOnDys01_task-SelfpacedHandTapL_acq-StimOffPreDopa_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
U

...

Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-EL010_ses-EcogLfpMedOffOnDys01_task-SelfpacedHandTapB_acq-StimOffDopa70_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Us

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-EL010_ses-EcogLfpMedOffOnDys01_task-Free_acq-StimOffDopa80_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annotation

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_35760/2753054600.py:14: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(ra, bids_path=bids_path, format="BrainVision", allow_preload=True)


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-EL010\ses-EcogLfpMedOffOnDys01\sub-EL010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-EL010_ses-EcogLfpMedOffOnDys01_task-Rest_acq-StimOffDopa85_run-1_ieeg.vhdr.
Channels marked as bad:
['ACC_L_X_D2_TM']
Channels marked as bad:
none


In [5]:
#Richard's code

for ra, task, acquisition in zip(raws, annot_table['task'], annot_table['acquisition']):
    print(ra.info)
    bids_path = BIDSPath(
        subject=subject,
        session="EcogLfpMedOffOnDys01",
        task=task,
        acquisition=acquisition,
        run=run,
        suffix=suffix,
        datatype=datatype,
        root=r"C:\Users\Jonathan\Documents\DATA\Dyskinesia",
    )
    write_raw_bids(
        ra, bids_path=bids_path, format="BrainVision", allow_preload=True
    )

<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\README'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annotations descriptions: ['RestStimOffPreDopa']
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\ieeg\sub-010_ses-EcogLfpMedOffOnDys01_task-Rest_acq-StimOffPreDopa_run-1_events.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\dataset_description.json'...
Writ

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-010_ses-EcogLfpMedOffOnDys01_task-Rest_acq-StimOffPreDopa_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annotations descrip

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-010_ses-EcogLfpMedOffOnDys01_task-SelfpacedHandTapL_acq-StimOffPreDopa_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annota

...

Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-010_ses-EcogLfpMedOffOnDys01_task-SelfpacedHandTapB_acq-StimOffDopa70_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annotat

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-010_ses-EcogLfpMedOffOnDys01_task-Free_acq-StimOffDopa80_run-1_ieeg.vhdr.
<Info | 12 non-empty values
 bads: []
 ch_names: LFP_R_01_STN_MT, LFP_R_02_STN_MT, LFP_R_03_STN_MT, ...
 chs: 16 DBS, 6 ECoG, 2 EEG, 2 EMG, 1 ECG, 7 misc
 custom_ref_applied: False
 description: Anonymized using a time shift to preserve age at acquisition
 dig: 22 items (22 EEG)
 experimenter: mne_anonymize
 highpass: 0.0 Hz
 line_freq: 50.0
 lowpass: 2000.0 Hz
 meas_date: 2022-02-08 10:53:06 UTC
 nchan: 34
 projs: []
 sfreq: 4000.0 Hz
 subject_info: 26 items (dict)
>
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.tsv'...
Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\participants.json'...
Used Annotations descript

C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Coordinate frame could not be inferred from the raw object and the BIDSPath.space was none, skipping the writing of channel positions
  write_raw_bids(
C:\Users\Jonathan\AppData\Local\Temp/ipykernel_1164/3384952987.py:15: RuntimeWarning: Converting data files to BrainVision format
  write_raw_bids(


Writing 'C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv'...
Wrote C:\Users\Jonathan\Documents\DATA\Dyskinesia\sub-010\ses-EcogLfpMedOffOnDys01\sub-010_ses-EcogLfpMedOffOnDys01_scans.tsv entry with ieeg\sub-010_ses-EcogLfpMedOffOnDys01_task-Rest_acq-StimOffDopa85_run-1_ieeg.vhdr.


In [9]:
i=0
annot_table['task'][i]

'Rest'